In [ ]:
import training
training.run_pipeline(
    "Dutch_F_Old_CorrieMulder",  # folder with audios | real path: /workspace/rvc-cli/datasets/model_name
    32,                 # CPU cores
    25,                  # save every x epoch
    250,                  # total epochs
    12,                 # batch size
    "/workspace/base/macsi1",     # real path: /workspace/rvc-cli/g_pretrained.pth
    save_only_latest=True,
    overtraining_detector=True, overtraining_threshold=50,
    skip_preprocessing=False, skip_extraction=False, skip_training=False
)

In [ ]:
import training
training.run_pipeline(
    "Dutch_F_Old_FrederiekVerschoor",  # folder with audios | real path: /workspace/rvc-cli/datasets/model_name
    32,                 # CPU cores
    25,                  # save every x epoch
    250,                  # total epochs
    12,                 # batch size
    "/workspace/base/macsi1",     # real path: /workspace/rvc-cli/g_pretrained.pth
    save_only_latest=True,
    overtraining_detector=True, overtraining_threshold=50,
    skip_preprocessing=False, skip_extraction=False, skip_training=False
)

In [ ]:
import training
training.run_pipeline(
    "Dutch_F_Old_JokevanWaardenburg",  # folder with audios | real path: /workspace/rvc-cli/datasets/model_name
    32,                 # CPU cores
    25,                  # save every x epoch
    250,                  # total epochs
    12,                 # batch size
    "/workspace/base/macsi1",     # real path: /workspace/rvc-cli/g_pretrained.pth
    save_only_latest=True,
    overtraining_detector=True, overtraining_threshold=50,
    skip_preprocessing=False, skip_extraction=False, skip_training=False
)

In [1]:
# Proof auth model == convert reference audio

import os
import inference

def automate_inference(input_audio, pitch, input_folder):
    # List all .pth files in the input folder
    model_files = [f for f in os.listdir(input_folder) if f.endswith('.pth')]
    
    # Iterate over each model file and run the inference pipeline
    for model_file in model_files:
        model_path = os.path.join(input_folder, model_file)
        output_audio = os.path.join(input_folder, f"{os.path.splitext(model_file)[0]}_{pitch}.wav")
        
        # Run the inference pipeline
        inference.run_pipeline(
            pitch,
            input_audio,
            output_audio,
            model_path
        )
        
        print(f"Processed {model_file} and saved output to {output_audio}")

# Example usage
input_audio = "/workspace/ds_valid/avm_validation/F_Modus.wav"
pitch = 0
input_folder = "/workspace/rvc-cli/logs/Dutch_F_Old_CorrieMulder"

automate_inference(input_audio, pitch, input_folder)


Output file saved to '/workspace/rvc-cli/logs/Dutch_F_Old_CorrieMulder/Dutch_F_Old_CorrieMulder_25e_1400s_0.wav'
Inference completed successfully!
Processed Dutch_F_Old_CorrieMulder_25e_1400s.pth and saved output to /workspace/rvc-cli/logs/Dutch_F_Old_CorrieMulder/Dutch_F_Old_CorrieMulder_25e_1400s_0.wav


In [3]:
import os
import re
import shutil
from pathlib import Path

def copy_best_epoch_files(
    project_name_start,
    work_folder='/workspace/rvc-cli/logs',
    output_folder='/workspace/weights/new_SVM_F_Old',
    overwrite=False,     # True -> overwrite existing; False -> add -copyN suffix
    keep_suffix=False,   # True -> keep _best/_best_epoch in filename
    dry_run=False        # True -> show what would happen without copying
):
    work = Path(work_folder)
    out = Path(output_folder)
    out.mkdir(parents=True, exist_ok=True)

    copied_files = []
    no_best_epoch_projects = []

    # Match both ..._best.pth and ..._best_epoch.pth
    pattern = re.compile(r'_best(?:_epoch)?\.pth$')

    for project_folder in os.listdir(work):
        if project_folder.startswith(project_name_start):
            project_path = work / project_folder
            if not project_path.is_dir():
                continue

            found_best_epoch = False

            for file_name in os.listdir(project_path):
                if pattern.search(file_name):
                    src_file = project_path / file_name

                    # New name: remove _best/_best_epoch unless keep_suffix=True
                    if keep_suffix:
                        new_file_name = file_name
                    else:
                        new_file_name = pattern.sub('.pth', file_name)

                    dest_file = out / new_file_name

                    # Handle duplicates
                    if dest_file.exists() and not overwrite:
                        base = dest_file.stem
                        ext = dest_file.suffix
                        i = 1
                        while True:
                            candidate = out / f"{base}-copy{i}{ext}"
                            if not candidate.exists():
                                dest_file = candidate
                                break
                            i += 1

                    if dry_run:
                        print(f"[DRY-RUN] Would copy: {src_file} -> {dest_file}")
                    else:
                        shutil.copy2(src_file, dest_file)
                        copied_files.append(str(dest_file))

                    found_best_epoch = True

            if not found_best_epoch:
                no_best_epoch_projects.append(project_folder)

    # Summary
    print("Copied files:")
    for f in copied_files:
        print(f)

    print("\nProject folders without *_best(.pth|_epoch.pth) files:")
    for project in no_best_epoch_projects:
        print(project)


# Example usage
project_name_start = "Dutch_F_Old_"
copy_best_epoch_files(project_name_start)


Copied files:
/workspace/weights/Dutch_F_Old_JokevanWaardenburg_50e_3100s.pth
/workspace/weights/Dutch_F_Old_FrederiekVerschoor_125e_9000s.pth
/workspace/weights/Dutch_F_Old_CorrieMulder_100e_5600s.pth

Project folders without *_best(.pth|_epoch.pth) files:
Dutch_F_Old_TruusEngelkamp
Dutch_F_Old_RiadeJong
Dutch_F_Old_MargjeStolte


In [4]:
# Blend all models between each other

import os
import blending

def blend_all_weights(input_folder, output_folder, ratios):
    weight_files = [f for f in os.listdir(input_folder) if f.endswith('.pth')]

    for i in range(len(weight_files)):
        for j in range(i + 1, len(weight_files)):
            file1 = weight_files[i]
            file2 = weight_files[j]

            for ratio in ratios:
                output_name = f"{os.path.splitext(file1)[0]}_{os.path.splitext(file2)[0]}_blend_{ratio}".replace(' ', '_')
                output_path = os.path.join(output_folder, output_name)
                blending.run_pipeline(
                    output_path,
                    os.path.join(input_folder, file1),
                    os.path.join(input_folder, file2),
                    ratio
                )

# Example usage
blend_all_weights(
    input_folder="/workspace/weights/new_SVM_F_Old",
    output_folder="/workspace/weights/new_SVM_F_Old",
    ratios=[0.3, 0.5, 0.7]
)

Running blending...
Blended file was saved to '/workspace/weights/new_SVM_F_Old/Dutch_F_Old_JokevanWaardenburg_50e_3100s_Dutch_F_Old_CorrieMulder_100e_5600s_blend_0.3.pth'
\Blending completed successfully!
Running blending...
Blended file was saved to '/workspace/weights/new_SVM_F_Old/Dutch_F_Old_JokevanWaardenburg_50e_3100s_Dutch_F_Old_CorrieMulder_100e_5600s_blend_0.5.pth'
\Blending completed successfully!
Running blending...
Blended file was saved to '/workspace/weights/new_SVM_F_Old/Dutch_F_Old_JokevanWaardenburg_50e_3100s_Dutch_F_Old_CorrieMulder_100e_5600s_blend_0.7.pth'
\Blending completed successfully!
Running blending...
Blended file was saved to '/workspace/weights/new_SVM_F_Old/Dutch_F_Old_JokevanWaardenburg_50e_3100s_Dutch_F_Old_FrederiekVerschoor_125e_9000s_blend_0.3.pth'
\Blending completed successfully!
Running blending...
Blended file was saved to '/workspace/weights/new_SVM_F_Old/Dutch_F_Old_JokevanWaardenburg_50e_3100s_Dutch_F_Old_FrederiekVerschoor_125e_9000s_blend_0

In [ ]:
# 1st blended model proof == convert reference audio

import os
import inference

def automate_inference(input_audio, pitch, input_folder):
    # Only use model files that include '_blend_' in their names
    model_files = [f for f in os.listdir(input_folder) if f.endswith('.pth') and '_blend_' in f]
    
    for model_file in model_files:
        model_path = os.path.join(input_folder, model_file)
        output_audio = os.path.join(input_folder, f"{os.path.splitext(model_file)[0]}_{pitch}.wav")
        
        inference.run_pipeline(
            pitch,
            input_audio,
            output_audio,
            model_path
        )
        
        print(f"Processed {model_file} and saved output to {output_audio}")

# Example usage
input_audio = "/workspace/ds_valid/avm_validation/F_Modus.wav"
pitch = 0
input_folder = "/workspace/weights/new_SVM_F_Old"

automate_inference(input_audio, pitch, input_folder)


In [5]:
# 2nd blended model proof == convert multiple validation sets

# Model file
# Input folder 1
# Input folder 2
# Input folder 3

!python3 /workspace/tools/batch_inference_for_multiple_input_folders.py \
    /workspace/weights/new_SVM_F_Old/Dutch_F_Old_JokevanWaardenburg_50e_3100s_Dutch_F_Old_FrederiekVerschoor_125e_9000s_blend_0.5.pth \
    /workspace/ds_valid/svm_F_validation/emo \
    /workspace/ds_valid/svm_F_validation/lang \
    /workspace/ds_valid/svm_F_validation/sing


Running batch inference...
Your files are waiting for you at directory 'Dutch_F_Old_JokevanWaardenburg_50e_3100s_Dutch_F_Old_FrederiekVerschoor_125e_9000s_blend_0.5_SVM_Valid/emo'

Batch inference completed successfully!
Processed /workspace/ds_valid/svm_F_validation/emo -> Dutch_F_Old_JokevanWaardenburg_50e_3100s_Dutch_F_Old_FrederiekVerschoor_125e_9000s_blend_0.5_SVM_Valid/emo
Running batch inference...
Your files are waiting for you at directory 'Dutch_F_Old_JokevanWaardenburg_50e_3100s_Dutch_F_Old_FrederiekVerschoor_125e_9000s_blend_0.5_SVM_Valid/lang'

Batch inference completed successfully!
Processed /workspace/ds_valid/svm_F_validation/lang -> Dutch_F_Old_JokevanWaardenburg_50e_3100s_Dutch_F_Old_FrederiekVerschoor_125e_9000s_blend_0.5_SVM_Valid/lang
Running batch inference...
Your files are waiting for you at directory 'Dutch_F_Old_JokevanWaardenburg_50e_3100s_Dutch_F_Old_FrederiekVerschoor_125e_9000s_blend_0.5_SVM_Valid/sing'

Batch inference completed successfully!
Processed /

In [ ]:
# SVM Modus definition
# >>> Giannotti, Kimberly
